<a href="https://colab.research.google.com/github/gustavsmnt/Classification-Analysis-using-CNN-and-LSTM-on-Wheezing-Sounds/blob/main/CNN_with_MFCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#IMPORT LIBRARY JALANKAN PERTAMA

In [ ]:
import numpy as np
import pandas as pd

import os
import librosa

import scipy
from scipy.stats import skew
from tqdm import tqdm, tqdm_pandas

tqdm.pandas()

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.svm import SVC

In [ ]:
# audio_train_files = os.listdir('../input/audio_train')
# audio_test_files = os.listdir('../input/audio_test')

# train = pd.read_csv('../input/train.csv')
# submission = pd.read_csv('../input/sample_submission.csv')

TRAIN_PATH_NORMAL = "/content/drive/MyDrive/dataset/datasetnormal.csv"
TRAIN_PATH_ABNORMAL = "/content/drive/MyDrive/dataset/datasetabnormal.csv"
TRAIN_NORMAL = "/content/drive/MyDrive/dataset/train/normal/"
TRAIN_ABNORMAL = "/content/drive/MyDrive/dataset/train/abnormal/"
TEST = "/content/drive/MyDrive/dataset/test"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_dataset_normal = pd.read_csv(TRAIN_PATH_NORMAL, sep=',')
df_dataset_abnormal = pd.read_csv(TRAIN_PATH_ABNORMAL, sep=',')

In [ ]:
df_dataset_normal

,fname,labels,labels_count,target_labels_count
0,102_1b1_Ar_sc_Meditron.wav,normal,1,1
1,121_1p1_Tc_sc_Meditron.wav,normal,1,1
2,122_2b1_Al_mc_LittC2SE.wav,normal,1,1
3,121_1b1_Tc_sc_Meditron.wav,normal,1,1
4,125_1b1_Tc_sc_Meditron.wav,normal,1,1
5,123_1b1_Al_sc_Meditron.wav,normal,1,1
6,126_1b1_Al_sc_Meditron.wav,normal,1,1
7,127_1b1_Ar_sc_Meditron.wav,normal,1,1
8,136_1b1_Ar_sc_Meditron.wav,normal,1,1
9,143_1b1_Al_sc_Meditron.wav,normal,1,1


In [ ]:
df_dataset_abnormal

,fname,labels,labels_count,target_labels_count
0,101_1b1_Al_sc_Meditron.wav,abnormal,0,0
1,103_2b2_Ar_mc_LittC2SE.wav,abnormal,0,0
2,104_1b1_Al_sc_Litt3200.wav,abnormal,0,0
3,104_1b1_Ar_sc_Litt3200.wav,abnormal,0,0
4,104_1b1_Ll_sc_Litt3200.wav,abnormal,0,0
5,104_1b1_Lr_sc_Litt3200.wav,abnormal,0,0
6,104_1b1_Pl_sc_Litt3200.wav,abnormal,0,0
7,104_1b1_Pr_sc_Litt3200.wav,abnormal,0,0
8,105_1b1_Tc_sc_Meditron.wav,abnormal,0,0
9,106_2b1_Pl_mc_LittC2SE.wav,abnormal,0,0


In [ ]:
#FITUR EKSTRAKSI JALANKAN KEDUA

In [ ]:
import librosa
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
from tqdm import tqdm

In [ ]:
SAMPLING_RATE = 44100
MFCC_NUM = 20
MFCC_MAX_LEN = 2500

target_labels = ['normal', 'abnormal']

In [ ]:
def wav2mfcc(wave, max_len=MFCC_MAX_LEN):
#     mfcc = librosa.feature.mfcc(wave, sr=16000)
    mfcc = librosa.feature.mfcc(wave, n_mfcc=MFCC_NUM, sr=SAMPLING_RATE)
    

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc

In [ ]:
def get_label_num(labels):
    lbs = labels.split(",")
#     target_lb = "Accelerating_and_revving_and_vroom"
    target_arr = np.zeros(2)
    for lb in lbs:
        if(lb in target_labels):
            i = target_labels.index(lb)
            target_arr[i] = 1
            break
    return target_arr

In [ ]:
X = []
y = []

def append_X_Y(labels, wave):
    y.append(get_label_num(labels))
    mfcc = wav2mfcc(wave)
    X.append(mfcc)

for index, row in tqdm(df_dataset_normal.iterrows()):
    labels = row["labels"]
    wave, sr = librosa.load(TRAIN_NORMAL + row["fname"], mono=True, sr=44100)
    wave = wave[::3]

    append_X_Y(labels, wave)

for index, row in tqdm(df_dataset_abnormal.iterrows()):
    labels = row["labels"]
    wave, sr = librosa.load(TRAIN_ABNORMAL + row["fname"], mono=True, sr=44100)
    wave = wave[::3]

    append_X_Y(labels, wave)

32it [00:02, 15.74it/s]
33it [00:07,  4.45it/s]


In [ ]:
X = np.array(X)
y = np.array(y)
X.shape[0] == len(y)

True

In [ ]:
X.shape

(65, 20, 2500)

In [ ]:
X[0].shape

(20, 2500)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=True)

In [ ]:
X_test

array([[[-3.0536887e+02, -2.9983499e+02, -3.0777789e+02, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 1.6413754e+02,  1.7290146e+02,  1.7005811e+02, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 1.2266220e+01,  1.6246042e+01,  2.0183327e+01, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        ...,
        [ 2.9403529e+00,  3.2168474e+00,  3.5231352e+00, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 7.5323019e+00,  3.2698550e+00,  4.8040152e-03, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 5.7676401e+00,  2.8255830e+00,  8.9652491e-01, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00]],

       [[-4.2518472e+02, -4.1062119e+02, -3.9805203e+02, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 1.4909970e+02,  1.5715771e+02,  1.6092545e+02, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 7.4910057e+01, 

In [ ]:
#MODEL CNN JALANKAN KETIGA

In [ ]:
# Feature dimension
feature_dim_1 = MFCC_NUM
# Second dimension of the feature is dim2
feature_dim_2 = MFCC_MAX_LEN
channel = 1
epochs = 100
batch_size = 2
verbose = 1
num_classes = len(target_labels)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = y_train
y_test_hot = y_test

In [ ]:
import keras
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from keras import optimizers

In [ ]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [ ]:
model = get_model()

optimizer = tensorflow.keras.optimizers.SGD(learning_rate=0.001, decay=1e-2, momentum=0.9, nesterov=True)
# optimizer = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])
model.fit(X_train, y_train_hot, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test_hot))

Epoch 1/100
2/2 [==============================] - 19s 8s/step - loss: 62.0847 - accuracy: 0.3846 - val_loss: 166.3151 - val_accuracy: 0.2308
Epoch 2/100
2/2 [==============================] - 15s 6s/step - loss: 67.5769 - accuracy: 0.4808 - val_loss: 63.8256 - val_accuracy: 0.2308
Epoch 3/100


In [ ]:
#MODEL TESTING JALANKAN KEEMPAT

In [ ]:
df_test = pd.read_csv("/content/drive/MyDrive/dataset/test.csv")
TEST = "/content/drive/MyDrive/dataset/test/"

for index, row in tqdm(df_test.iterrows()):
    wave, sr = librosa.load(TEST + row["fname"], mono=True, sr=None)
    wave = wave[::2]
    
    mfcc = wav2mfcc(wave)
    X_test = mfcc.reshape(1, feature_dim_1, feature_dim_2, channel)
    preds = model.predict(X_test)[0]
    
    for i, col in enumerate(target_labels):
        df_test.loc[index, col] = preds[i]

In [ ]:
df_test

In [ ]:
model.save('cnn_mfcc.h5')